1.	Project Setup & Connection

In [1]:
import uuid
import random
from datetime import datetime, timedelta
from pymongo import MongoClient
from random import choice, randint, uniform
from pprint import pprint


#Part 1
# Establishing connection
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']


print("MongoDB connection successful. Database:", db.name)

# --- DATABASE RESET ---

#--- Checks if the database exists and drop it to ensure a clean run. I implemented this to avoid duplicate key errors during repeated runs
#and to ensure that the database starts fresh each time the script is executed.

if 'eduhub_db' in client.list_database_names():
    client.drop_database('eduhub_db')
    print("Database 'eduhub_db' successfully dropped for clean re-run.")
else:
    print("Database 'eduhub_db' does not exist yet. Proceeding with creation.")


MongoDB connection successful. Database: eduhub_db
Database 'eduhub_db' successfully dropped for clean re-run.


1.2 Creating Collections


In [2]:
#schema validation. refers to 6-1
user_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"],
        "properties": {
            "email": {"bsonType": "string", "pattern": "^.+@.+\\..+$"},
            "role": {"enum": ["student", "instructor"]},
            "dateJoined": {"bsonType": "date"},
            "isActive": {"bsonType": "bool"}
        }
    }
}

# --- CREATE COLLECTION ---
# using a to Try/except block to catch error. it runs if the collection already exists
try:
    db.create_collection('users', validator=user_validator)
    print("Collection 'users' created with validation.")
except Exception as e:
    # If the collection exists, MongoDB raises an error on re-creation.
    print(f"Collection 'users' already exists or validation error: {e}")

# Create other  5 collections 
for col_name in ['courses', 'enrollments', 'lessons', 'assignments', 'submissions']:
    if col_name not in db.list_collection_names():
        db.create_collection(col_name)

Collection 'users' created with validation.


Part 2: Data Population

Task 2.1: Insert Sample Data
 generate and insert at least 20 users (students/instructors),
   8 courses, 15 enrollments, 25 lessons, 10 assignments, and 12 submissions.

In [3]:


#Part 2




#Step 1: Users Collection
#mix of students and instructors via genereating unique IDs

# Helper function to generate unique IDs
def generate_id():
    return str(uuid.uuid4())[:8] 

users_data = []
instructor_ids = []
student_ids = []
user_count = 20

# Create Instructor Users (4 users)
for i in range(4):
    instructor_id = generate_id()
    instructor_ids.append(instructor_id)
    users_data.append({
        "userId": instructor_id,
        "email": f"instructor{i+1}@eduhub.com",
        "firstName": f"InstName{i+1}",
        "lastName": f"InstLast{i+1}",
        "role": "instructor",
        "dateJoined": datetime.now() - timedelta(days=randint(365, 730)), 
        "profile": {"bio": f"Experienced instructor in Subject {i+1}.", "skills": ["Python", "SQL", "Data Analysis"]},
        "isActive": True
    })

# Create Student Users (16 users)
for i in range(16):
    student_id = generate_id()
    student_ids.append(student_id)
    users_data.append({
        "userId": student_id,
        "email": f"student{i+1}@eduhub.com",
        "firstName": f"StudName{i+1}",
        "lastName": f"StudLast{i+1}",
        "role": "student",
        "dateJoined": datetime.now() - timedelta(days=randint(30, 365)), # Joined 1 month - 1 year ago
        "profile": {"bio": f"Eager to learn Subject {randint(1,4)}.", "skills": ["Javascript", "HTML", "CSS"]},
        "isActive": True
    })

# Insert into collection
db.users.insert_many(users_data)
print(f"Inserted {len(users_data)} users.")








Inserted 20 users.


Task 2, Step 2 to 6: VALIDATIONS

In [4]:
#Step 2: Courses Collection
#Courses need an instructorId from the list generated.

course_data = []
course_ids = []
categories = ["Programming", "Design", "Business", "Marketing"]

for i in range(8): #looping through 8 courses
    course_id = generate_id()
    course_ids.append(course_id)
    instructor_id = choice(instructor_ids) # Assigning a random instructor
    
    course_data.append({
        "courseId": course_id,
        "title": f"The Ultimate Course in {choice(categories)} {i+1}",
        "description": f"Learn everything about {categories[i % len(categories)]}!",
        "instructorId": instructor_id, # <--- Reference to users.userId
        "category": categories[i % len(categories)],
        "level": choice(["beginner", "intermediate", "advanced"]),
        "duration": randint(5, 50),
        "price": round(uniform(49.99, 199.99), 2),
        "tags": [categories[i % len(categories)].lower(), "2024", "online"],
        "createdAt": datetime.now() - timedelta(days=randint(30, 300)),
        "updatedAt": datetime.now(),
        "isPublished": choice([True, True, False]) 
    })

db.courses.insert_many(course_data)
print(f"Inserted {len(course_data)} courses.")


#Step 3: Enrollments Collection
#This links students to courses. At least 15 enrollments.


enrollment_data = []
course_student_pairs = set()
enrollment_count = 0


while enrollment_count < 15: # Ensure at least 15 unique enrollments
    student_id = choice(student_ids) # Random student
    course_id = choice(course_ids) # Random course
    
    # Ensures a student doesn't enroll in the same course twice
    if (student_id, course_id) not in course_student_pairs: #loop checks for unique pairs
        course_student_pairs.add((student_id, course_id)) #then adds it to the set
        enrollment_count += 1 # Increment counts only on unique enrollment
        
        enrollment_data.append({ #enrollment data to be looped through
            "enrollmentId": generate_id(),
            "studentId": student_id, # <--- Reference to users.userId
            "courseId": course_id,   # <--- Reference to courses.courseId
            "enrollmentDate": datetime.now() - timedelta(days=randint(7, 90)),
            "completionStatus": choice(["in_progress", "completed", "in_progress"]),
            "lastAccessed": datetime.now() - timedelta(hours=randint(1, 48)),
            "progressPercentage": randint(0, 100)
        })

db.enrollments.insert_many(enrollment_data)
print(f"Inserted {len(enrollment_data)} enrollments.")


#Step 4: Lessons Collection
#Lessons belong to courses. We need at least 25 lessons total.


lesson_data = [] #empty list to hold lesson data
lesson_map = {} # Map courseId to a list of lessonIds

for course_id in course_ids: #loop to go through each course

    # Each course gets 3-5 lessons
    num_lessons = randint(3, 5)
    lesson_map[course_id] = []
    for i in range(num_lessons): #loop to create lessons for each course
        lesson_id = generate_id() #generate unique lesson ID
        lesson_map[course_id].append(lesson_id) # Add to map for reference
        
        lesson_data.append({
            "lessonId": lesson_id,
            "courseId": course_id, # <--- Reference to courses.courseId
            "title": f"Lesson {i+1}: Introduction to MongoDB and PyMongo",
            "contentLink": f"/content/{course_id}/{lesson_id}.mp4",
            "order": i + 1,
            "createdAt": datetime.now() - timedelta(days=randint(50, 200)),
            "isPublished": True
        })

db.lessons.insert_many(lesson_data)
print(f"Inserted {len(lesson_data)} lessons.")


#Step 5: Assignments Collection
#Assignments belong to courses (and optionally lessons). We need at least 10 assignments.

# Each assignment is linked to a course and optionally to a lesson within that course.
#empty list to hold assignment data and IDs
assignment_data = [] 
assignment_ids = []

for i in range(10): #loop to create 10 assignments
    course_id = choice(course_ids) # Random course for each assignment
    assignment_id = generate_id() #generate unique assignment ID
    assignment_ids.append(assignment_id) # Store for reference
    
    # Link to a specific lesson in the course (if any exist)
    lesson_id = choice(lesson_map.get(course_id, [None])) 
    
    assignment_data.append({
        "assignmentId": assignment_id,
        "courseId": course_id, # <--- Reference to courses.courseId
        "lessonId": lesson_id, # <--- Reference to lessons.lessonId (optional)
        "title": f"Project {i+1} - SQL Data Warehousing",
        "description": "Data Modeling and Medallion Architecture.",
        "maxGrade": 100, 
        "dueDate": datetime.now() + timedelta(days=randint(1, 14)), # Due in the next 1 to 14 days
        "createdAt": datetime.now() - timedelta(days=randint(10, 50)) # Created 10-50 days ago
    })

db.assignments.insert_many(assignment_data)
print(f"Inserted {len(assignment_data)} assignments.")


#Step 6: Submissions Collection
#Submissions link students to assignments. We need at least 12 submissions.

#empoty lists to hold submission data. Count starts from 0 because we will increment it
submission_data = []
submission_count = 0

# Use a subset of students and assignments for submissions
for assignment_id in random.sample(assignment_ids, k=6): # Select 6 random assignments. k here is a keyword argument
    for student_id in random.sample(student_ids, k=2):     # Assign 2 random students to each
        
        # We need the courseId from the assignment to ensure the student is enrolled 
        
    
        submission_count += 1
        if submission_count > 12:
            break # Break inner loop if we exceed 12 submissions. 

        submission_data.append({
            "submissionId": generate_id(),
            "assignmentId": assignment_id, # <--- Reference to assignments.assignmentId
            "studentId": student_id,       # <--- Reference to users.userId
            "submissionDate": datetime.now() - timedelta(days=randint(1, 7)),
            "submissionContent": f"Submitted file link for {assignment_id}",
            "grade": randint(60, 100) if choice([True, False]) else None, # Some graded, some not
            "feedback": "Great work on the aggregation!" if choice([True, False]) else None,
            "isGraded": True if choice([True, False]) else False
        })
    if submission_count > 12: 
        break # Break outer loop if we exceed 12 submissions. 

db.submissions.insert_many(submission_data)
print(f"Inserted {len(submission_data)} submissions.")


#Task 2.2: Data Relationships
#Verification


print("\n--- Verification Counts ---")
print(f"Total Users: {db.users.count_documents({})}")
print(f"Total Courses: {db.courses.count_documents({})}")
print(f"Total Enrollments: {db.enrollments.count_documents({})}")
print(f"Total Lessons: {db.lessons.count_documents({})}")
print(f"Total Assignments: {db.assignments.count_documents({})}")
print(f"Total Submissions: {db.submissions.count_documents({})}")


Inserted 8 courses.
Inserted 15 enrollments.
Inserted 31 lessons.
Inserted 10 assignments.
Inserted 12 submissions.

--- Verification Counts ---
Total Users: 20
Total Courses: 8
Total Enrollments: 15
Total Lessons: 31
Total Assignments: 10
Total Submissions: 12


Part 3: CRUD Operations

Task 3.1: Create Operations (Insert)

This involves inserting new documents into the collections.

In [5]:


#Task 3.1: Create Operations (Insert)
#
#a. Add a new student user

# Generate a new unique ID for the student
new_student_id = str(uuid.uuid4())[:8]

new_student = {
    "userId": new_student_id,
    "email": "new.student@example.com",
    "firstName": "Chiamaka",
    "lastName": "Adams",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {"bio": "Just joined to learn Financial Analysis.", "skills": ["Finance"]},
    "isActive": True
}

# Insert the document
result = db.users.insert_one(new_student)
print(f"New student added with _id: {result.inserted_id}")
student_ids.append(new_student_id) # Add to tracking list

#b. Create a new course

# Generate a new unique ID for the course
new_course_id = str(uuid.uuid4())[:8]
instructor_for_new_course = choice(instructor_ids) # Picks a random instructor

new_course = {
    "courseId": new_course_id,
    "title": "Introduction to Financial Engineering with Python",
    "description": "A deep dive into Financial Data Modelling and Python.",
    "instructorId": instructor_for_new_course,
    "category": "Programming",
    "level": "intermediate",
    "duration": 40,
    "price": 149.99,
    "tags": ["Finance", "data-analysis", "python"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": False # Not published yet
}

result = db.courses.insert_one(new_course)
print(f"New course created with _id: {result.inserted_id}")
course_ids.append(new_course_id) # Appended to  tracking list


#c. Enroll a student in a course
#enrolling the new student just created (new_student_id) into the new course (new_course_id).


new_enrollment = {
    "enrollmentId": generate_id(),
    "studentId": new_student_id,      # Reference:  new student
    "courseId": new_course_id,        # Reference:  new course
    "enrollmentDate": datetime.now(),
    "completionStatus": "in_progress",
    "lastAccessed": datetime.now(),
    "progressPercentage": 0
}

result = db.enrollments.insert_one(new_enrollment)
print(f"New enrollment created with _id: {result.inserted_id}")

#d. Add a new lesson to an existing course
# adding a lesson to the new course created (new_course_id).


new_lesson = {
    "lessonId": generate_id(),
    "courseId": new_course_id, # Reference: The new course
    "title": "Lesson 1: PyMongo Connection and CRUD",
    "contentLink": f"/content/{new_course_id}/lesson_3.mp4",
    "order": 1,
    "createdAt": datetime.now(),
    "isPublished": True
}

result = db.lessons.insert_one(new_lesson)
print(f"New lesson added with _id: {result.inserted_id}")




New student added with _id: 68e112ac1ad8bd3de67db742
New course created with _id: 68e112ac1ad8bd3de67db743
New enrollment created with _id: 68e112ac1ad8bd3de67db744
New lesson added with _id: 68e112ac1ad8bd3de67db745


Task 3.2: Read Operations

---Querying uses the find() and find_one() methods with various query operators.---

In [6]:


#a. Finding all active students
#This query uses a simple equality match on the role and isActive fields.


active_students = db.users.find({
    "role": "student", 
    "isActive": True
}, {"firstName": 1, "lastName": 1, "email": 1, "_id": 0}) # Projection to select specific fields

print("\n--- Active Students (5 samples) ---")
pprint(list(active_students.limit(5)))

#b. Retrieve course details with instructor information
#an aggregation operation

#using new course we created for a specific example
target_course_id = new_course_id

pipeline = [
    # 1. Match operator for the specific course
    {"$match": {"courseId": target_course_id}},
    # 2. Join (lookup) the instructor details from the 'users' collection
    {"$lookup": {
        "from": "users",          # The collection to join
        "localField": "instructorId", # Field from the 'courses' collection
        "foreignField": "userId",     # Field from the 'users' collection
        "as": "instructor_details"    # The name of the new array field
    }},

    # 3. Unwind the instructor_details array 
    {"$unwind": "$instructor_details"},

    # 4. select the desired output fields, with project operator
    {"$project": {
        "_id": 0,
        "courseTitle": "$title",
        "category": 1,
        "price": 1,
        "instructorName": {"$concat": ["$instructor_details.firstName", " ", "$instructor_details.lastName"]},
        "instructorEmail": "$instructor_details.email"
    }}
]

course_with_instructor = db.courses.aggregate(pipeline)

print("\n--- Course Details with Instructor ---")
pprint(list(course_with_instructor))

#c. Get all courses in a specific category

target_category = "Programming"

programming_courses = db.courses.find({
    "category": target_category
}, {"courseId": 1, "title": 1, "price": 1, "_id": 0})

print(f"\n--- Courses in the '{target_category}' Category ---")
pprint(list(programming_courses))

#d. Find students enrolled in a particular course
#We need to query the collection for enrollments(gives us stidents details) and then use the resulting studentIds to look up the user details.

# Pick a popular course ID from the list
popular_course_id = choice(course_ids[:-1]) 
course_title = db.courses.find_one({"courseId": popular_course_id})['title']

# Aggregation Pipeline to find enrolled students
pipeline = [
    {"$match": {"courseId": popular_course_id}},
    {"$lookup": {
        "from": "users",
        "localField": "studentId",
        "foreignField": "userId",
        "as": "student_info"
    }},
    {"$unwind": "$student_info"},
    {"$project": {
        "_id": 0,
        "studentName": {"$concat": ["$student_info.firstName", " ", "$student_info.lastName"]},
        "enrollmentDate": 1,
        "progressPercentage": 1
    }}
]

enrolled_students = db.enrollments.aggregate(pipeline)

print(f"\n--- Students Enrolled in '{course_title}' ---")
pprint(list(enrolled_students))

#e. Search courses by title (case-insensitive, partial match)
#This uses, $regex, and the options flag, $options: 'i' (for case-insensitive).

search_term = "ultimate"

search_query = {
    "title": {
        "$regex": search_term, 
        "$options": "i" # 'i' stands for case-insensitive
    }
}

matching_courses = db.courses.find(search_query, {"title": 1, "category": 1, "_id": 0})

print(f"\n--- Courses Matching '{search_term}' (case-insensitive) ---")
pprint(list(matching_courses))


--- Active Students (5 samples) ---
[{'email': 'student1@eduhub.com',
  'firstName': 'StudName1',
  'lastName': 'StudLast1'},
 {'email': 'student2@eduhub.com',
  'firstName': 'StudName2',
  'lastName': 'StudLast2'},
 {'email': 'student3@eduhub.com',
  'firstName': 'StudName3',
  'lastName': 'StudLast3'},
 {'email': 'student4@eduhub.com',
  'firstName': 'StudName4',
  'lastName': 'StudLast4'},
 {'email': 'student5@eduhub.com',
  'firstName': 'StudName5',
  'lastName': 'StudLast5'}]

--- Course Details with Instructor ---
[{'category': 'Programming',
  'courseTitle': 'Introduction to Financial Engineering with Python',
  'instructorEmail': 'instructor2@eduhub.com',
  'instructorName': 'InstName2 InstLast2',
  'price': 149.99}]

--- Courses in the 'Programming' Category ---
[{'courseId': '459585d4',
  'price': 165.59,
  'title': 'The Ultimate Course in Marketing 1'},
 {'courseId': '8a5889fb',
  'price': 93.07,
  'title': 'The Ultimate Course in Business 5'},
 {'courseId': '68aabd8e',
  '

Task 3.3: Update Operations

This uses the update_one() or update_many() methods along with $set (to replace a field's value) and $push (to add an element to an array).

In [7]:


#a. Update a user’s profile information. update the new student created.


# Use $set to update multiple fields
update_result = db.users.update_one(
    {"userId": new_student_id},
    {"$set": {
        "profile.bio": "Advanced MongoDB user now!",
        "profile.avatar": "new_avatar.jpg",
        "updatedAt": datetime.now() #datetime.now returns real-time date& time
    }}
)
print(f"\nUpdated User Profile: Matched {update_result.matched_count}, Modified {update_result.modified_count}")

#b. Mark a course as published

#updating the new course created.

update_result = db.courses.update_one(
    {"courseId": new_course_id},
    {"$set": {
        "isPublished": True,
        "updatedAt": datetime.now()
    }}
)
print(f"Marked Course as Published: Matched {update_result.matched_count}, Modified {update_result.modified_count}")

#c. Update assignment grades
#finding an 'un-graded submission' and update its grade.

# Find the first submission that is not yet graded
submission_to_grade = db.submissions.find_one({"isGraded": False})

if submission_to_grade:
    submission_id = submission_to_grade['submissionId']
    
    update_result = db.submissions.update_one(
        {"submissionId": submission_id},
        {"$set": {
            "grade": 88,
            "feedback": "Good attempt!",
            "isGraded": True
        }}
    )
    print(f"Updated Submission Grade for ID {submission_id}: Modified {update_result.modified_count}")
else:
    print("Could not find an un-graded submission to update.")

#d. Add tags to an existing course
#This uses the $push operator to add an item to an array field.

course_to_tag = choice(course_ids) # this picks ick a random course
update_result = db.courses.update_one(
    {"courseId": course_to_tag},
    {"$push": {
        "tags": "new-curriculum"
    }}
)
print(f"Added tag to course {course_to_tag}: Modified {update_result.modified_count}")


Updated User Profile: Matched 1, Modified 1
Marked Course as Published: Matched 1, Modified 1
Updated Submission Grade for ID 7c783f48: Modified 1
Added tag to course 7cf94a2f: Modified 1


Task 3.4: Delete Operations

#Deletion uses update_one() for soft deletes and delete_one() for hard deletes.

In [8]:


#a. Remove a user (soft delete by setting isActive to false)
#softly "delete" the student just created.


update_result = db.users.update_one(
    {"userId": new_student_id},
    {"$set": {
        "isActive": False,
        "deactivatedAt": datetime.now()
    }}
)
print(f"\nSoft Deleted User: Matched {update_result.matched_count}, Modified {update_result.modified_count}")

#b. Delete an enrollment (Hard Delete)


delete_result = db.enrollments.delete_one(
    {"studentId": new_student_id, "courseId": new_course_id}
)
print(f"Deleted Enrollment: {delete_result.deleted_count} document(s) deleted.")

#c. Remove a lesson from a course (Hard Delete)


delete_result = db.lessons.delete_one(
    {"lessonId": new_lesson['lessonId']}
)
print(f"Deleted Lesson: {delete_result.deleted_count} document(s) deleted.")



Soft Deleted User: Matched 1, Modified 1
Deleted Enrollment: 1 document(s) deleted.
Deleted Lesson: 1 document(s) deleted.


Part 4: Advanced Queries and Aggregation

In [9]:
#Task 4.1: Complex Queries
#1. Find courses with price between $50 and $200

#This uses the Range Query Operators: $gte (greater than or equal to) and $lte (less than or equal to).


price_range_courses = db.courses.find({
    "price": {
        "$gte": 50.00, # Greater than or equal to $50
        "$lte": 200.00 # Less than or equal to $200
    },
    "isPublished": True # Filter for published courses
}, {"courseId": 1, "title": 1, "price": 1, "_id": 0})

print("\n--- Courses Priced Between $50 and $200 (Published) ---")
pprint(list(price_range_courses.limit(5)))



--- Courses Priced Between $50 and $200 (Published) ---
[{'courseId': '459585d4',
  'price': 165.59,
  'title': 'The Ultimate Course in Marketing 1'},
 {'courseId': 'a9169110',
  'price': 75.55,
  'title': 'The Ultimate Course in Programming 2'},
 {'courseId': 'e4747adc',
  'price': 167.12,
  'title': 'The Ultimate Course in Programming 3'},
 {'courseId': '7cf94a2f',
  'price': 55.77,
  'title': 'The Ultimate Course in Programming 4'},
 {'courseId': '32c40f7f',
  'price': 172.29,
  'title': 'The Ultimate Course in Marketing 6'}]


2. Get users who joined in the last 6 months

In [10]:
#This uses date arithmetic, specifically the $gte operator 


six_months_ago = datetime.now() - timedelta(days=6 * 30) # Approximate 6 months

recent_users = db.users.find({
    "dateJoined": {
        "$gte": six_months_ago # Joined date must be GREATER than or equal to 6 months ago
    },
    "isActive": True
}, {"userId": 1, "email": 1, "dateJoined": 1, "_id": 0})

print("\n--- Users who joined in the last 6 months (Active) ---")
pprint(list(recent_users.limit(5)))



--- Users who joined in the last 6 months (Active) ---
[{'dateJoined': datetime.datetime(2025, 4, 26, 13, 27, 21, 362000),
  'email': 'student2@eduhub.com',
  'userId': '90feb3c4'},
 {'dateJoined': datetime.datetime(2025, 9, 4, 13, 27, 21, 362000),
  'email': 'student3@eduhub.com',
  'userId': 'be491b59'},
 {'dateJoined': datetime.datetime(2025, 8, 3, 13, 27, 21, 362000),
  'email': 'student5@eduhub.com',
  'userId': 'ed6b5095'},
 {'dateJoined': datetime.datetime(2025, 7, 27, 13, 27, 21, 362000),
  'email': 'student6@eduhub.com',
  'userId': 'ff374bfb'},
 {'dateJoined': datetime.datetime(2025, 5, 22, 13, 27, 21, 362000),
  'email': 'student7@eduhub.com',
  'userId': '6645c9fe'}]


3. Find courses that have specific tags using $in operator

In [11]:

#The $in operator checks if an array field contains any of the specified values.


tags = ["python", "Finanical Engineering", "2024"]

tagged_courses = db.courses.find({
    "tags": {
        "$in": tags # Finds documents where 'tags' array contains any of the target_tags
    }
}, {"courseId": 1, "title": 1, "tags": 1, "_id": 0})

print(f"\n--- Courses Tagged with {tags} ---")
pprint(list(tagged_courses.limit(5)))



--- Courses Tagged with ['python', 'Finanical Engineering', '2024'] ---
[{'courseId': '459585d4',
  'tags': ['programming', '2024', 'online'],
  'title': 'The Ultimate Course in Marketing 1'},
 {'courseId': 'a9169110',
  'tags': ['design', '2024', 'online'],
  'title': 'The Ultimate Course in Programming 2'},
 {'courseId': 'e4747adc',
  'tags': ['business', '2024', 'online'],
  'title': 'The Ultimate Course in Programming 3'},
 {'courseId': '7cf94a2f',
  'tags': ['marketing', '2024', 'online', 'new-curriculum'],
  'title': 'The Ultimate Course in Programming 4'},
 {'courseId': '8a5889fb',
  'tags': ['programming', '2024', 'online'],
  'title': 'The Ultimate Course in Business 5'}]


4. Retrieve assignments with due dates in the next week

In [12]:
#$gte and $lte to define a time window.


now = datetime.now()
next_week = now + timedelta(days=7) #TIMEDELTA creates a time difference of 7 days

upcoming_assignments = db.assignments.find({
    "dueDate": {
        "$gte": now,        # Due date is NOW or later
        "$lte": next_week   # Due date is up to 7 days from NOW
    }
}, {"assignmentId": 1, "title": 1, "dueDate": 1, "_id": 0})

print("\n--- Assignments Due in the Next Week ---")
pprint(list(upcoming_assignments))



--- Assignments Due in the Next Week ---
[{'assignmentId': 'e8930341',
  'dueDate': datetime.datetime(2025, 10, 10, 13, 27, 22, 7000),
  'title': 'Project 1 - SQL Data Warehousing'},
 {'assignmentId': '74a4f5c1',
  'dueDate': datetime.datetime(2025, 10, 9, 13, 27, 22, 22000),
  'title': 'Project 3 - SQL Data Warehousing'},
 {'assignmentId': '5c3bc090',
  'dueDate': datetime.datetime(2025, 10, 9, 13, 27, 22, 22000),
  'title': 'Project 4 - SQL Data Warehousing'},
 {'assignmentId': 'd12f0bfa',
  'dueDate': datetime.datetime(2025, 10, 7, 13, 27, 22, 22000),
  'title': 'Project 5 - SQL Data Warehousing'},
 {'assignmentId': '8d0fff33',
  'dueDate': datetime.datetime(2025, 10, 10, 13, 27, 22, 23000),
  'title': 'Project 6 - SQL Data Warehousing'},
 {'assignmentId': '2e413b30',
  'dueDate': datetime.datetime(2025, 10, 7, 13, 27, 22, 23000),
  'title': 'Project 9 - SQL Data Warehousing'}]


Task 4.2: AGGREGATION PIPELINE

In [13]:
#1. Course Enrollment Statistics
#------Count total enrollments per course


pipeline_enrollment_stats = [
    # 1. Group by courseId and count the enrollments

    {"$group": {
        "_id": "$courseId",
        "totalEnrollments": {"$sum": 1} # $sum: 1 counts each document in the group
    }},

    # 2. lookup the course title. Lookup operatot is similar to join in SQL. here, we are joining 
    # the enrollments collection with the courses collection
    {"$lookup": {
        "from": "courses",
        "localField": "_id",
        "foreignField": "courseId",
        "as": "course_info"
    }},

    # 3. Unwind the course info. unwind operator does the opposite of group. here, 
    # it deconstructs the array field from the previous lookup stage, so that we can access individual fields
    {"$unwind": "$course_info"},

    # 4. Project the final output structure

    {"$project": { #project operator is used to specify which fields to include or exclude in the output documents
        "_id": 0,
        "courseId": "$_id",
        "courseTitle": "$course_info.title",
        "totalEnrollments": 1
    }},
    # 5. Sort by enrollment count (descending)
    {"$sort": {"totalEnrollments": -1}}
]

course_enrollment_stats = list(db.enrollments.aggregate(pipeline_enrollment_stats))
print("\n--- Course Enrollment Statistics (Top 3) ---")
pprint(course_enrollment_stats[:3])





--- Course Enrollment Statistics (Top 3) ---
[{'courseId': '8a5889fb',
  'courseTitle': 'The Ultimate Course in Business 5',
  'totalEnrollments': 4},
 {'courseId': '459585d4',
  'courseTitle': 'The Ultimate Course in Marketing 1',
  'totalEnrollments': 3},
 {'courseId': '97cf5d11',
  'courseTitle': 'The Ultimate Course in Marketing 8',
  'totalEnrollments': 2}]


CATEGORY GROUP PIPELINE

In [14]:
#Group by course category
#This aggregates the enrollment count from the previous step and groups that result by category.


pipeline_category_group = [
    # (Starting from a collection that has course details, e.g., courses)
    # 1. Look up enrollments for each course (JOIN)
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseId",
        "foreignField": "courseId",
        "as": "enrollments"
    }},
    # 2. Add a field for the enrollment count
    {"$addFields": {
        "enrollmentCount": {"$size": "$enrollments"}
    }},
    # 3. Group by category and sum up the counts
    {"$group": {
        "_id": "$category",
        "totalCategoryEnrollments": {"$sum": "$enrollmentCount"},
        "numberOfCourses": {"$sum": 1}
    }},
    # 4. Sort by total enrollments (descending)
    {"$sort": {"totalCategoryEnrollments": -1}},
    {"$project": {"_id": 0, "category": "$_id", "totalCategoryEnrollments": 1, "numberOfCourses": 1}}
]

category_enrollment_stats = list(db.courses.aggregate(pipeline_category_group))
print("\n--- Enrollment Grouped by Category ---")
pprint(category_enrollment_stats)



--- Enrollment Grouped by Category ---
[{'category': 'Programming',
  'numberOfCourses': 3,
  'totalCategoryEnrollments': 7},
 {'category': 'Business', 'numberOfCourses': 2, 'totalCategoryEnrollments': 4},
 {'category': 'Marketing', 'numberOfCourses': 2, 'totalCategoryEnrollments': 3},
 {'category': 'Design', 'numberOfCourses': 2, 'totalCategoryEnrollments': 1}]


4.2.2 STUDENT PERFORMANCE ANALYSIS

In [15]:
#Student Performance Analysis
#aggregration pipeline for the Average grade per student

pipeline_avg_grade = [
    # 1. Match only graded submissions. so that we exclude ungraded ones

    {"$match": {"isGraded": True, "grade": {"$ne": None}}},
    
    # 2. Group by studentId and calculate the average grade
    {"$group": {
        "_id": "$studentId",
        "averageGrade": {"$avg": "$grade"},
        "totalSubmissions": {"$sum": 1}
    }},
    # 3. Lookup student name
    {"$lookup": {
        "from": "users",
        "localField": "_id",
        "foreignField": "userId",
        "as": "student_info"
    }},
    # 4. Unwind and Project
    {"$unwind": "$student_info"},
    {"$project": {
        "_id": 0,
        "studentName": {"$concat": ["$student_info.firstName", " ", "$student_info.lastName"]},
        "averageGrade": {"$round": ["$averageGrade", 2]} # Round to 2 decimal places
    }},
    # 5. Sort by average grade (descending)
    {"$sort": {"averageGrade": -1}}
]

student_avg_grades = list(db.submissions.aggregate(pipeline_avg_grade))
print("\n--- Student Average Grades (Top 3) ---")
pprint(student_avg_grades[:3])



--- Student Average Grades (Top 3) ---
[{'averageGrade': 88.0, 'studentName': 'StudName12 StudLast12'},
 {'averageGrade': 84.0, 'studentName': 'StudName5 StudLast5'},
 {'averageGrade': 82.0, 'studentName': 'StudName15 StudLast15'}]


3. INSTRUCTOR ANALYTICS

In [16]:
#Total students taught by each instructor
#This requires joining courses (via instructorId) → enrollments (via courseId) → then getting the unique studentIds and count.


pipeline_instructor_students = [
    # 1. Match only published courses

    {"$match": {"isPublished": True}},

    # 2. Group by instructor
    {"$group": {
        "_id": "$instructorId",
        "courseIds": {"$push": "$courseId"}
    }},

    # 3. Look up enrollments using the courseIds array, so that we can get all enrollments for those courses.
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseIds", # Array of course IDs
        "foreignField": "courseId",
        "as": "all_enrollments"
    }},

    # 4. Get unique student IDs from all enrollments
    {"$unwind": "$all_enrollments"},
    {"$group": {
        "_id": "$_id", # Group back by instructorId
        "uniqueStudentIds": {"$addToSet": "$all_enrollments.studentId"} # $addToSet gets unique elements
    }},

    # 5. Count the unique students
    {"$project": {
        "_id": 0,
        "instructorId": "$_id",
        "totalStudentsTaught": {"$size": "$uniqueStudentIds"}
    }},

    # 6. Lookup instructor name. so that we can display the instructor's name with the count
    {"$lookup": {
        "from": "users",
        "localField": "instructorId",
        "foreignField": "userId",
        "as": "instructor_info"
    }},
    {"$unwind": "$instructor_info"},
    {"$project": {
        "instructorName": {"$concat": ["$instructor_info.firstName", " ", "$instructor_info.lastName"]},
        "totalStudentsTaught": 1
    }},
    {"$sort": {"totalStudentsTaught": -1}}
]

instructor_student_count = list(db.courses.aggregate(pipeline_instructor_students))
print("\n--- Instructor Total Students Taught ---")
pprint(instructor_student_count)



--- Instructor Total Students Taught ---
[{'instructorName': 'InstName3 InstLast3', 'totalStudentsTaught': 4},
 {'instructorName': 'InstName1 InstLast1', 'totalStudentsTaught': 3},
 {'instructorName': 'InstName2 InstLast2', 'totalStudentsTaught': 1}]


4.2.3 REVENUE GENERATED BY INSTRUCTIOR


In [17]:
#Revenue generated per instructor
#This assumes that revenue = price * enrollment_count. 
# we can implement this by joining courses → enrollments → grouping by instructorId and summing up the revenue.


pipeline_instructor_revenue = [
    # 1. Lookup enrollments (starting from courses)
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseId",
        "foreignField": "courseId",
        "as": "enrollments"
    }},

    # 2. Add a field for the total revenue generated by this course
    {"$addFields": {
        "courseRevenue": {"$multiply": ["$price", {"$size": "$enrollments"}]}
    }},

    # 3. Group by instructorId and sum the course revenue
    {"$group": {
        "_id": "$instructorId",
        "totalRevenue": {"$sum": "$courseRevenue"}
    }},

    # 4. Lookup instructor name
    {"$lookup": {
        "from": "users",
        "localField": "_id",
        "foreignField": "userId",
        "as": "instructor_info"
    }},

    # 5. Project and format
    {"$unwind": "$instructor_info"},
    {"$project": {
        "_id": 0,
        "instructorName": {"$concat": ["$instructor_info.firstName", " ", "$instructor_info.lastName"]},
        "totalRevenue": {f"$round": ["$totalRevenue", 2]}
    }},
    {"$sort": {"totalRevenue": -1}}
]

instructor_revenue = list(db.courses.aggregate(pipeline_instructor_revenue))
print("\n--- Instructor Total Revenue (Simulated) ---")
pprint(instructor_revenue)



--- Instructor Total Revenue (Simulated) ---
[{'instructorName': 'InstName1 InstLast1', 'totalRevenue': 869.05},
 {'instructorName': 'InstName3 InstLast3', 'totalRevenue': 717.45},
 {'instructorName': 'InstName4 InstLast4', 'totalRevenue': 133.5},
 {'instructorName': 'InstName2 InstLast2', 'totalRevenue': 55.77}]


PART 5: INDEXING AND PERFORMANCE

In [18]:
#pipeline for monthly enrollment trends

pipeline_monthly_trends = [
    # 1. Project a new field for the month and year of enrollment , so that we can group by these fields 
    {"$project": {
        "month": {"$month": "$enrollmentDate"},
        "year": {"$year": "$enrollmentDate"}
    }},
    # 2. Group by year and month, count enrollments, so that we can get the total enrollments per month
    {"$group": {
        "_id": {"year": "$year", "month": "$month"},
        "totalEnrollments": {"$sum": 1} # Count each enrollment
    }},
    # 3. Sort by year and month 
    {"$sort": {
        "_id.year": 1,
        "_id.month": 1
    }},
    # 4. Project final output format. Project opoerator is used to specify which fields to include or exclude in the output 
    {"$project": {
        "_id": 0,
        "year": "$_id.year",
        "month": "$_id.month",
        "totalEnrollments": 1
    }}
]

print("\n--- Monthly Enrollment Trends ---")
monthly_trends = list(db.enrollments.aggregate(pipeline_monthly_trends))    
pprint(monthly_trends)




--- Monthly Enrollment Trends ---
[{'month': 7, 'totalEnrollments': 3, 'year': 2025},
 {'month': 8, 'totalEnrollments': 3, 'year': 2025},
 {'month': 9, 'totalEnrollments': 9, 'year': 2025}]


4.2.4 Most popular course categories

In [19]:
#pipeline for most popular course categoreies

most_popular_course_categories = [
    # 1. Lookup enrollments to get enrollment counts per course
    {"$lookup": {
        "from": "enrollments",
        "localField": "courseId",
        "foreignField": "courseId",
        "as": "enrollments"
    }},
    # 2. Add a field for enrollment count
    {"$addFields": {
        "enrollmentCount": {"$size": "$enrollments"}
    }},
    # 3. Group by category and sum enrollment counts
    {"$group": {
        "_id": "$category",
        "totalEnrollments": {"$sum": "$enrollmentCount"},
        "numberOfCourses": {"$sum": 1}
    }},
    # 4. Sort by total enrollments (descending)
    {"$sort": {"totalEnrollments": -1}},
    # 5. Project final output format
    {"$project": {
        "_id": 0,
        "category": "$_id",
        "totalEnrollments": 1,
        "numberOfCourses": 1
    }}
]

print("\n--- Most Popular Course Categories ---")
popular_categories = list(db.courses.aggregate(most_popular_course_categories))
pprint(popular_categories)


--- Most Popular Course Categories ---
[{'category': 'Programming', 'numberOfCourses': 3, 'totalEnrollments': 7},
 {'category': 'Business', 'numberOfCourses': 2, 'totalEnrollments': 4},
 {'category': 'Marketing', 'numberOfCourses': 2, 'totalEnrollments': 3},
 {'category': 'Design', 'numberOfCourses': 2, 'totalEnrollments': 1}]


4.2.4 Student engagement metrics

In [20]:
#pipeline for student engagement metrics. 
#we'll get the average progress percentage of students across all their enrollments.

student_eng_metrics = [
    # 1. Group by studentId to calculate average progress
    {"$group": {
        "_id": "$studentId",
        "averageProgress": {"$avg": "$progressPercentage"},
        "totalCoursesEnrolled": {"$sum": 1}
    }},
    # 2. get student name. (lookup)
    {"$lookup": {
        "from": "users",
        "localField": "_id",
        "foreignField": "userId",
        "as": "student_info"
    }},

    # 3. Unwind and project
    {"$unwind": "$student_info"},
    {"$project": {
        "_id": 0,
        "studentName": {"$concat": ["$student_info.firstName", " ", "$student_info.lastName"]},
        "averageProgress": {"$round": ["$averageProgress", 2]},
        "totalCoursesEnrolled": 1
    }},

    # 4. Sort by average progress (descending)
    {"$sort": {"averageProgress": -1}}
]

print("\n--- Student Engagement Metrics ---")
engagement_metrics = list(db.enrollments.aggregate(student_eng_metrics))
pprint(engagement_metrics[:5]) # Display top 5 students by average progress


--- Student Engagement Metrics ---
[{'averageProgress': 83.0,
  'studentName': 'StudName9 StudLast9',
  'totalCoursesEnrolled': 1},
 {'averageProgress': 77.0,
  'studentName': 'StudName14 StudLast14',
  'totalCoursesEnrolled': 2},
 {'averageProgress': 74.0,
  'studentName': 'StudName15 StudLast15',
  'totalCoursesEnrolled': 1},
 {'averageProgress': 60.5,
  'studentName': 'StudName13 StudLast13',
  'totalCoursesEnrolled': 2},
 {'averageProgress': 44.0,
  'studentName': 'StudName7 StudLast7',
  'totalCoursesEnrolled': 1}]


Part 5: INDEXING AND PERFORMANCE

In [22]:
# Task 5.1: Index Creation

# 1. User email lookup (Unique index). unique indexes are faster for lookups and ensure no duplicate emails

db.users.create_index([("email", 1)], unique=True)
print("Index created on users.email") 

# 2. Course search by title and category (Compound index) compound indexes are useful for queries that filter on multiple fields
db.courses.create_index([("title", 1), ("category", 1)])
print("Index created on courses.title and courses.category")

# 3. Assignment queries by due date
db.assignments.create_index([("dueDate", 1)])
print("Index created on assignments.dueDate")

# 4. Enrollment queries by student and course (Compound index)
db.enrollments.create_index([("studentId", 1), ("courseId", 1)])
print("Index created on enrollments.studentId and enrollments.courseId")

print("\nAll required indexes created.")

Index created on users.email
Index created on courses.title and courses.category
Index created on assignments.dueDate
Index created on enrollments.studentId and enrollments.courseId

All required indexes created.


Task 5.2: Query Optimization

In [ ]:
import time
#1. Analyze query performance using explain() method in PyMongo
sample_query = db.users.find({"email": "eduhub_mongodb_project.ipynb"})
explain_plan = sample_query.explain()
print("\n--- Explain Plan for Sample Query ---")
pprint(explain_plan)




--- Explain Plan for Sample Query ---
{'command': {'$db': 'eduhub_db',
             'filter': {'email': 'eduhub_mongodb_project.ipynb'},
             'find': 'users'},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'docsExamined': 0,
                                        'indexName': 'email_1',
                                        'isCached': False,
                                        'keyPattern': '{ email: 1 }',
                                        'keysExamined': 0,
                                        'nReturned': 0,
                                        'stage': 'EXPRESS_IXSCAN'},
                    'executionSuccess': True,
                    'executionTimeMillis': 1,
                    'nReturned': 0,
                    'totalDocsExamined': 0,
                    'totalKeysExamined': 0},
 'explainVersion': '1',
 'ok': 1.0,
 'queryPlanner': {'indexFilterSet': False,
                  'maxIndexedAndSolutionsReached': Fals

5.2.2 Optimize at least 3 slow queries

In [25]:
#2. Optimize at least 3 slow queries
# Query 1: Find user by email (should be fast because of unique index)
start_time = time.time()
user = db.users.find_one({"email": "eduhub_mongodb_project.ipynb"})
end_time = time.time()
print(f"\nQuery 1 Execution Time: {end_time - start_time:.6f} seconds")
pprint(user)


Query 1 Execution Time: 0.004932 seconds
None


5.2.3 Document the performance improvements using Python timing functions

In [27]:
import time
from pprint import pprint

#Document the performance improvements using Python timing functions
#using a previously created pipeline



# package the aggregation command, including the pipeline, into a dictionary.
explain_command = {
    'aggregate': 'enrollments', # The collection to aggregate on
    'pipeline': pipeline_enrollment_stats,
    'cursor': {}, 
    'explain': True 
}


start_time = time.time()

execution_plan = db.command(explain_command)
end_time = time.time()

# --- 3. Displaying the Results ---
print("\n--- Optimized Query Analysis (Monthly Enrollment Stats Aggregation) ---")
print(f"Execution Time (PyMongo/Python): {(end_time - start_time) * 1000:.4f} ms")

# The execution time from the MongoDB engine is located deep in the output
try:
    mongo_time = execution_plan['stages'][-1]['$cursor']['executionStats']['executionTimeMillis']
    print(f"Execution Time (MongoDB Engine): {mongo_time} ms")
except KeyError:
    # Fallback if the structure is different (sometimes happens with smaller pipelines)
    print("Could not extract specific MongoDB engine time from explain output structure.")


pprint(execution_plan)


--- Optimized Query Analysis (Monthly Enrollment Stats Aggregation) ---
Execution Time (PyMongo/Python): 253.1514 ms
Could not extract specific MongoDB engine time from explain output structure.
{'command': {'$db': 'eduhub_db',
             'aggregate': 'enrollments',
             'cursor': {},
             'explain': True,
             'lsid': {'id': Binary(b'\x90X\x97\x97\x8c\xc4DX\x95\x9a\xa4A\xe0\xd8\x18\xd0', 4)},
             'pipeline': [{'$group': {'_id': '$courseId',
                                      'totalEnrollments': {'$sum': 1}}},
                          {'$lookup': {'as': 'course_info',
                                       'foreignField': 'courseId',
                                       'from': 'courses',
                                       'localField': '_id'}},
                          {'$unwind': '$course_info'},
                          {'$project': {'_id': 0,
                                        'courseId': '$_id',
                                 